In [1]:
from colbert.infra import ColBERTConfig
from colbert.modeling.checkpoint import Checkpoint

ckpt = Checkpoint(
    "colbert-ir/colbertv2.0", colbert_config=ColBERTConfig(root="experiments")
)

[Dec 09, 23:19:52] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/workspace/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/workspace/.venv/lib/python3.12/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [2]:
passage = [
    "Alan Mathison Turing was an English mathematician, computer scientist, logician, cryptanalyst, philosopher and theoretical biologist."
]

In [4]:
vectors = ckpt.docFromText(passage)[0]

In [5]:
vectors.shape

torch.Size([27, 128])

In [6]:
query_vectors = ckpt.queryFromText(["Who was Alan Turing?"])[0]
query_vectors.shape


#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: Who was Alan Turing?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2040,  2001,  5070, 28639,  1029,   102,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



torch.Size([32, 128])

In [7]:
import numpy as np
import torch
from binascii import hexlify
from typing import Dict, List


def binarize_token_vectors_hex(vectors: torch.Tensor) -> Dict[str, str]:
    binarized_token_vectors = np.packbits(np.where(vectors > 0, 1, 0), axis=1).astype(
        np.int8
    )
    vespa_token_feed = dict()
    for index in range(0, len(binarized_token_vectors)):
        vespa_token_feed[index] = str(
            hexlify(binarized_token_vectors[index].tobytes()), "utf-8"
        )
    return vespa_token_feed


def float_query_token_vectors(vectors: torch.Tensor) -> Dict[str, List[float]]:
    vespa_token_feed = dict()
    for index in range(0, len(vectors)):
        vespa_token_feed[index] = vectors[index].tolist()
    return vespa_token_feed


In [8]:
import json

print(json.dumps(binarize_token_vectors_hex(vectors)))
print(json.dumps(float_query_token_vectors(query_vectors)))

{"0": "0378450025c4c6a188208143dc4496f4", "1": "9628898060c4c621306081439c6096f4", "2": "e4ac49ee71c4cd28a07904189d44d1c4", "3": "f3284b844140e0cb0c407d0418c0bb65", "4": "21ca71184ddcd403e0b82829da6635dd", "5": "03ea49a149cd5580871911445a408fe4", "6": "93b88e0061c6c7a128a090439fabd6f4", "7": "90388e0061c6c7ad28a081439f2bd6f4", "8": "16b9416000c46c25106ac9738bc8e4f4", "9": "96b85e1e20c4c4ed18c8d1c1d8cb99f5", "10": "00000000000000000000000000000000", "11": "82790399204667e958c9c3479b96f8bc", "12": "1218961e24d6c468398880c3bdcb9cf5", "13": "00000000000000000000000000000000", "14": "1631da1124d6e74119c8d3c59afebf7d", "15": "9719ce1000d6e42d30c890c9fda990f5", "16": "00000000000000000000000000000000", "17": "372bc221a5446320480ad7468bb2b7f5", "18": "03358b1424d64f210988d745dcbb9175", "19": "1710d6142cd6e621398890c1fde894f5", "20": "00000000000000000000000000000000", "21": "9235961c20d6c64d3dc8d6f1bcfb92f5", "22": "93388e0060d6c6ad28a080c39debd4f4", "23": "1239ce1e2456c4450988d0c1b5ea9ef5", "